In [2]:
import uproot
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from tqdm.auto import tqdm # Progressbar

%matplotlib widget
import matplotlib.pyplot as plt

# Config
# 40 GeV 744339 744340
# 50 GeV 744237 744242 744246 744326 744327 744328 744331
# 60 GeV 744249 744254 744258 744317 744323
# 70 GeV 744318 744323

run_number = 744318
clock_cut = 8192 # in clock, used for ecal bcid correction
ecal_start = 2488 # "True" start of ecal bcid data -> See with ecal people why event before this bcid are garbage
bin_size = 1 # In clock
triggers = 0 # Set to 0 to run on all trigger

# Read data directly from eos via xroot, you need to be identified with a kerberos5 token to use this feature
use_xroot = True
xroot_url = 'root://eosuser.cern.ch/'

# common_file_name = '/eos/user/a/apingaul/CALICE/Data/SPS_09_2018/Trivent/Common_{}.root'.format(run_number)
common_file_name = '/eos/user/a/apingaul/CALICE/Data/SPS_09_2018/Trivent/Common_{}_bcid_corrected.root'.format(run_number)
common_tree_name = 'common'

# Try recorrecting ecal_bcid without touching at bcid outside window [ecal_start, ecal_start+clock_cut]

if use_xroot:
    common_file_name = xroot_url + common_file_name


### Read the common tree

In [3]:
# Read the full tree
df_common = uproot.open(common_file_name)[common_tree_name].pandas.df()

# Or apply a selection  
# ecal_bcid_cut = 'ecal_nhit_slab>3 and (ecal_bcid>={} or ecal_bcid<={}) and sdhcal_EvtRevBcid>{} and sdhcal_EvtRevBcid<={}'.format(ecal_start,ecal_start+clock_cut, clock_cut,clock_cut*2)
# df_common = uproot.open(common_file_name)[common_tree_name].pandas.df().query(ecal_bcid_cut)


### Some plotting function to use later

In [6]:
# Display peaks infos
# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def draw_arrow_between_peaks(ax, prev_x, x, y, arrow_ypos, distance_ypos):
    # Arrow
    ann_list = []
    ann_list.append(ax.annotate('', xy=(prev_x, arrow_ypos),
                              xycoords='data', xytext=(x, arrow_ypos),
                              arrowprops=dict(arrowstyle='<->',color='r')))

    # Distance between peaks, write in the center of the arrow
    display(abs(int(prev_x-x)))
    ann_list.append(
        ax.annotate(abs(int(prev_x-x)), xy=((prev_x+x)/2, distance_ypos),
                    ha='center', xycoords='data', color='r')
    )
    return ann_list
    
# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def draw_peaks(hist=None, bins=None, ax=None, peaks_args=None):
    ''' hist, bins are the two first element returned by plt.hist()
        ax the axe used to plot the hist
        draw and return a list of annotations containing
            - The peaks marker
            - Their value
            - The arrows between the peaks
            - THe distance between the peaks
    '''
    if not peaks_args:
        peaks_args={
            'prominence': 100,
            'distance': 500
        }
    hist_peaks, _ = find_peaks(hist, **peaks_args)
    peak_xy_list = [bins[hist_peaks], hist[hist_peaks]]

    # Keep track of the annotations so we can delete them easily
    # Peak symbol
    ann_list = []
    ann_list.append(ax.plot(peak_xy_list[0], peak_xy_list[1], 'rv'))

    # Tweak positions on the canvas
    max_peak = peak_xy_list[1].max()
    print(max_peak)
    arrow_pos= max_peak * (0.6)
    distance_pos= max_peak * (0.3)
    prev_x = None
    for x, y in zip(peak_xy_list[0],peak_xy_list[1]):
        # Peak value
        ann_list.append(
            ax.annotate(int(x), xy=(x,y*1.2), textcoords='data', ha='center')
        )
        if prev_x:
            # arrow+distance between peaks
            ann_list.append(draw_arrow_between_peaks(ax, prev_x, x, y, arrow_pos, distance_pos))
        
        display((int(x), int(y)))
        prev_x = x
    return ann_list

# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def draw_vlines_and_arrows(lines_x=None, lines_y=None, ax=None, show_xpos=True):
    ''' Plot vertical lines + distance between them
        show_xpos to display the x_value of the vlines
    '''
    with plt.xkcd(.5, 100, 2):
        ann_list = []
        ann_list.append(ax.vlines(x=lines_x, ymin=0, ymax=lines_y, colors='r', linestyles='dashed'))
        prev_x = None
        for x in lines_x:
            if show_xpos:
                ann_list.append(ax.annotate(int(x), xy=(x, lines_y), textcoords='data', ha='center', color='r'))
            # Arrow between lines
            if prev_x:
                ann_list.extend(draw_arrow_between_peaks(ax, prev_x, x, lines_y, arrow_ypos=lines_y*0.95, distance_ypos=lines_y*0.9))
            prev_x = x
        return ann_list

# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def resize_labplot(fig, nrows=1, height=800, dpi=96, top_adjust=0.9):
    '''Small hack to properly resize the plot on jupiterlab
    '''
    # Need to readjust top of the plot to accomodate for a title with the tight_layout
    fig.tight_layout()
    fig.subplots_adjust(top=top_adjust)

    # Hack to force jupyterlab to properly size the plots
    # fig.canvas.layout.width = str(ncols*w/dpi)
    fig.canvas.layout.height = str(nrows*height/dpi)+ 'in'
    fig.show()

### BCID Distribution

In [7]:
# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def plot_bcid_distributions(fig=None, ax=None, left=None, right=None, low_cut=None, mid_cut=None, high_cut=None, file_title=None, canvas_options=None, hist_options=None):
    left_hist, left_bins, _ = ax[0].hist([
        df_common.query(low_cut)[left],
        df_common.query(mid_cut)[left],
        df_common.query(high_cut)[left]
        ],
        **hist_options
    )    

    right_hist, right_bins, _ = ax[1].hist([
        df_common.query(low_cut)[right],
        df_common.query(mid_cut)[right],
        df_common.query(high_cut)[right]
        ],
        **hist_options
    )

    # Show the legend on the right plot only
    with plt.xkcd(.5, 100, 2):
        ax[1].legend(loc='upper right', fontsize='xx-small')

    # Comment resize_labplot if not running on jupiterlab
    resize_labplot(fig, nrows=canvas_options['nrows'], height=h, dpi=canvas_options['dpi'])
    fig.savefig('./Figs/{file_title}_{run_number}.png'.format(file_title=file_title, run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")
    return((left_hist, left_bins), (right_hist, right_bins))

# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
plt.close('all') # Clean the list of figures, handy when calling multiple times this cell

# Define canvas/ hist option
w, h, dpi = 800, 400, 96
canvas_options = {
    'nrows' : 1,
    'ncols': 2,
    'dpi' : dpi,
    'sharey': True,
    'sharex': True,
}
canvas_options['figsize'] = canvas_options['ncols']*w/canvas_options['dpi'], canvas_options['nrows']*h/canvas_options['dpi']

# Choose a reasonable amount of bin to plot the histograms so we don't murder the ram
total_bins_bcid = abs(df_common['sdhcal_EvtBcid'].min())+abs(df_common['sdhcal_EvtBcid'].max())
# Precise binning is not necessary for this one
hist_bins_bcid = int(total_bins_bcid/(bin_size * 10))


# Cuts on the bcid
low_cut='sdhcal_EvtRevBcid<{}'.format(clock_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{}'.format(clock_cut,clock_cut*2)
high_cut='sdhcal_EvtRevBcid>={}'.format(clock_cut*2)

hist_options = {
    'bins': hist_bins_bcid,
    'alpha': .8,
    'color': ['b','g','r'],
    'histtype': 'stepfilled',
    'label': [low_cut, mid_cut, high_cut], # For the legend
    'stacked': True,
    'log': False
}


# Draw the distrib
# Normal ecal bcid
with plt.xkcd(.5, 100, 2):
    fig_bcid, ax_bcid = plt.subplots(**canvas_options)
    ax_bcid[0].set(xlabel='ecal_bcid (clock)')
    ax_bcid[1].set(xlabel='sdhcal_revbcid (clock)')
    fig_bcid.suptitle('BCID distribution in both calorimeters (Stacked histograms) - Run={}'.format(run_number))
(ecal_bcid_hist, ecal_bcid_hist_bins),_ = plot_bcid_distributions(fig_bcid, ax_bcid, 'ecal_bcid', 'sdhcal_EvtRevBcid', low_cut, mid_cut, high_cut, 'bcid', canvas_options, hist_options)

# Draw vlines on the ecal side
lines_x = [ecal_start, ecal_start + clock_cut]
lines_y = ecal_bcid_hist[0].max() * 3
annotations = draw_vlines_and_arrows(lines_x, lines_y, ax_bcid[0])
fig_bcid.savefig('./Figs/bcid_vlines_{run_number}.png'.format(run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")

# Corrected ecal bcid
with plt.xkcd(.5, 100, 2):
    fig_bcid_corrected, ax_bcid_corrected = plt.subplots(**canvas_options)
    ax_bcid_corrected[0].set(xlabel='ecal_bcid corrected (clock)')
    ax_bcid_corrected[1].set(xlabel='sdhcal_revbcid (clock)')
    fig_bcid_corrected.suptitle('corrected BCID distribution in both calorimeters (Stacked histograms) - Run={}'.format(run_number))
(ecal_bcid_corrected_hist, ecal_bcid_corrected_hist_bins),_ = plot_bcid_distributions(fig_bcid_corrected, ax_bcid_corrected, 'ecal_bcid_corrected', 'sdhcal_EvtRevBcid', low_cut, mid_cut, high_cut, 'bcid_corrected', canvas_options, hist_options)

# Draw vlines on the ecal_bcid distribution
lines_x = [ecal_start] * 4
for i in range(len(lines_x)):
    lines_x[i] += i * clock_cut 
lines_y = ecal_bcid_corrected_hist[0].max()
annotations.extend(draw_vlines_and_arrows(lines_x, lines_y, ax_bcid_corrected[0],show_xpos=False))
fig_bcid_corrected.savefig('./Figs/bcid_corrected_vlines_{run_number}.png'.format(run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8192

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2488, 10680, 18872, 27064]


8192

8192

8192

In [ ]:
# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# Plot only weird ecal bcid evt
with plt.xkcd(.5, 100, 2):
    fig_bcid_fringe, ax_bcid_fringe = plt.subplots(**canvas_options)
    ax_bcid_fringe[0].set(xlabel='ecal_bcid (clock)')
    ax_bcid_fringe[1].set(xlabel='ecal_bcid corrected (clock)')
    fig_bcid_fringe.suptitle('BCID distribution in both calorimeters (Stacked histograms) - Run={}'.format(run_number))

(low_bcid, high_bcid) = (ecal_start, ecal_start + clock_cut)
ecal_bcid_fringe_cut='and (ecal_bcid<{} or ecal_bcid>{})'.format(low_bcid, high_bcid)
low_cut='sdhcal_EvtRevBcid<{} {}'.format(clock_cut, ecal_bcid_fringe_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{} {}'.format(clock_cut,clock_cut*2, ecal_bcid_fringe_cut)
high_cut='sdhcal_EvtRevBcid>={} {}'.format(clock_cut*2, ecal_bcid_fringe_cut)
hist_options['label'] = [low_cut, mid_cut, high_cut]

plot_bcid_distributions(fig_bcid_fringe, ax_bcid_fringe, 'ecal_bcid', 'ecal_bcid_corrected', low_cut, mid_cut, high_cut, 'bcid_fringe', canvas_options, hist_options)

# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# Plot only proper ecal bcid evt
with plt.xkcd(.5, 100, 2):
    fig_bcid_core, ax_bcid_core = plt.subplots(**canvas_options)
    ax_bcid_core[0].set(xlabel='ecal_bcid (clock)')
    ax_bcid_core[1].set(xlabel='ecal_bcid corrected (clock)')
    fig_bcid_core.suptitle('BCID distribution in both calorimeters (Stacked histograms) - Run={}'.format(run_number))

ecal_bcid_core_cut='and (ecal_bcid>={} and ecal_bcid<={})'.format(low_bcid, high_bcid)
low_cut='sdhcal_EvtRevBcid<{} {}'.format(clock_cut, ecal_bcid_core_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{} {}'.format(clock_cut, clock_cut*2, ecal_bcid_core_cut)
high_cut='sdhcal_EvtRevBcid>={} {}'.format(clock_cut*2, ecal_bcid_core_cut)
hist_options['label'] = [low_cut, mid_cut, high_cut]

plot_bcid_distributions(fig_bcid_core, ax_bcid_core, 'ecal_bcid', 'ecal_bcid_corrected', low_cut, mid_cut, high_cut, 'bcid_core', canvas_options, hist_options)

In [ ]:
# Remove all annotation, handy to tweak the style without recomputing the whole hist
def remove_annotations(ann_list):
    for ann in ann_list:
        if isinstance(ann, list):
            remove_annotations(ann)
        else:
            ann.remove() # remove annotation itself
    # empty the list
    ann_list[:]=[]
    
display(annotations)
# remove_annotation(annotations)

# Plot $\Delta_{BCID}$

In [ ]:
plt.close('all')
def plot_delta_bcid(data_serie, bin_size=3, w=2000, h=800, dpi=96):
    total_bins = abs(data_serie.max()) + abs(data_serie.min())
    hist_bins = int(total_bins/(bin_size))

    with plt.xkcd(.5, 100, 2):
        fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)
        fig.suptitle('$\Delta_{BCID} (ecal - sdhcal)$' + ' - Run={}'.format(run_number))
        ax.set(xlabel='$\Delta_{BCID}$')

    # Plot the distribution
    hist, bins = np.histogram(data_serie, bins=hist_bins)
    plt.step(x=bins[:-1], y=hist, where='pre') # Don't forget to remove the overflow bin
    plt.yscale('log')

    # Plot the peaks position
#     with plt.xkcd():
#         corrected_peak_options = {
#             'prominence': 60,
#             'distance': 5000/bin_size
#         }
#         draw_peaks(hist=hist, bins=bins, ax=ax, peaks_args=corrected_peak_options)

    branch_name = data_serie.name
    fig.savefig('./Figs/{}_{}.png'.format(branch_name, run_number), transparent=False, dpi=dpi, bbox_inches="tight")

# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# plot_delta_bcid(df_common['delta_revbcid'])
plot_delta_bcid(df_common['delta_revbcid_corrected'])


In [ ]:
peak_ann=[]
display(peak_ann)
remove_annotations(peak_ann)
display(peak_ann)
corrected_peak_options = {
    'prominence': 50,
    'distance': 800
}

with plt.xkcd():
    peak_ann = draw_peaks(hist=deltabcid_hist, bins=deltabcid_hist_bins, ax=ax_deltabcid, peaks_args=corrected_peak_options)
fig_deltabcid.canvas.draw()



In [ ]:
def plot_delta_bcid_distributions(fig, ax, delta_revbcid, delta_revbcid_corrected, low_cut, mid_cut, high_cut, canvas_options=None, hist_options=None, file_title=None, save_plot=True):
    ax[0].hist([
            df_common.query(low_cut)[delta_revbcid],
            df_common.query(mid_cut)[delta_revbcid],
            df_common.query(high_cut)[delta_revbcid]
        ],
        **hist_options
    )

    # Draw hist with cuts for visualisation
    ax[1].hist([
            df_common.query(low_cut)[delta_revbcid_corrected],
            df_common.query(mid_cut)[delta_revbcid_corrected],
            df_common.query(high_cut)[delta_revbcid_corrected]
        ],
        **hist_options
    )

    # Show legend on right plot
    with plt.xkcd(.5, 100, 2):
        ax[1].legend(loc='lower right', fontsize='xx-small')

    # Comment resize_labplot if not running on jupiterlab
    resize_labplot(fig, nrows=canvas_options['nrows'], height=h, dpi=canvas_options['dpi'])
    if save_plot:
        fig.savefig('./Figs/{file_title}_{run_number}.png'.format(file_title=file_title, run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")
        # fig.savefig('./Figs/{file_title}_{run_number}.pdf'.format(file_title=file_title, run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")

# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
plt.close('all')
low_cut='sdhcal_EvtRevBcid<{}'.format(clock_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{}'.format(clock_cut,clock_cut*2)
high_cut='sdhcal_EvtRevBcid>={}'.format(clock_cut*2)
hist_options['label'] = [low_cut, mid_cut, high_cut]

total_bins = abs(df_common['delta_revbcid'].min())+abs(df_common['delta_revbcid'].max())
hist_bins = int(total_bins/bin_size) 
hist_options['bins'] = hist_bins
hist_options['log'] = True

with plt.xkcd(.5, 100, 2):
    fig_deltabcid_cuts, ax_deltabcid_cuts = plt.subplots(**canvas_options)
    fig_deltabcid_cuts.suptitle('$\Delta_{BCID} (ecal - sdhcal)$' + ' distribution (Stacked histograms) - Run={}'.format(run_number))
    ax_deltabcid_cuts[0].set(xlabel='delta_bcid (clock)')
    ax_deltabcid_cuts[1].set(xlabel='delta_bcid corrected (clock)')
plot_delta_bcid_distributions(fig_deltabcid_cuts, ax_deltabcid_cuts, 'delta_revbcid', 'delta_revbcid_corrected', low_cut, mid_cut, high_cut, canvas_options, hist_options, save_plot=False)

### Delta Revbcid
# compute full hist without cut for peak analysis but don't display it
delta_revbcid_hist, nbins_delta_revbcid, _ = ax_deltabcid_cuts[0].hist(
    df_common['delta_revbcid'],
    bins=hist_options['bins'],
    log=hist_options['log'],
    visible=False
)

delta_revbcid_corrected_hist, nbins_delta_revbcid_corrected, _ = ax_deltabcid_cuts[1].hist(
    df_common['delta_revbcid_corrected'],
    bins=hist_options['bins'],
    log=hist_options['log'],
    visible=False
)

peak_annotations=[]
with plt.xkcd():
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_hist, bins=nbins_delta_revbcid, ax=ax_deltabcid_cuts[0]))
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_corrected_hist, bins=nbins_delta_revbcid_corrected, ax=ax_deltabcid_cuts[1]))

fig_deltabcid_cuts.savefig('./Figs/delta_bcid_corrected_{}.png'.format(run_number), transparent=False, dpi=300, bbox_inches="tight")
# fig_deltabcid.savefig('./Figs/delta_bcid_{}.pdf'.format(run_number), transparent=False, dpi=300, bbox_inches="tight")

In [ ]:
with plt.xkcd(.5, 100, 2):
    fig_deltabcid_cuts_core, ax_deltabcid_cuts_core = plt.subplots(**canvas_options)
    fig_deltabcid_cuts_core.suptitle('$\Delta_{BCID} (ecal - sdhcal)$' + ' distribution (Stacked histograms) - Run={}'.format(run_number))
    ax_deltabcid_cuts_core[0].set(xlabel='delta_bcid corrected (clock)')
    ax_deltabcid_cuts_core[1].set(xlabel='delta_bcid corrected (clock)')
    

# Draw fringe
ecal_bcid_fringe_cut='and (ecal_bcid<{} or ecal_bcid>{})'.format(low_bcid, high_bcid)
low_cut='sdhcal_EvtRevBcid<{} {}'.format(clock_cut, ecal_bcid_fringe_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{} {}'.format(clock_cut,clock_cut*2, ecal_bcid_fringe_cut)
high_cut='sdhcal_EvtRevBcid>={} {}'.format(clock_cut*2, ecal_bcid_fringe_cut)
hist_options['label'] = [low_cut, mid_cut, high_cut]
ax_deltabcid_cuts_core[0].hist([
    df_common.query(low_cut)['delta_revbcid_corrected'],
    df_common.query(mid_cut)['delta_revbcid_corrected'],
    df_common.query(high_cut)['delta_revbcid_corrected']
    ],
    **hist_options
)
delta_revbcid_fringe_hist, nbins_delta_revbcid_fringe, _ = ax_deltabcid_cuts_core[0].hist(
    df_common.query(ecal_bcid_fringe_cut[4:])['delta_revbcid_corrected'],
    bins=hist_options['bins'],
    log=hist_options['log'],
    visible=False
)

# Draw core
ecal_bcid_core_cut='and (ecal_bcid>={} and ecal_bcid<={})'.format(low_bcid, high_bcid)
low_cut='sdhcal_EvtRevBcid<{} {}'.format(clock_cut, ecal_bcid_core_cut)
mid_cut='sdhcal_EvtRevBcid>={} and sdhcal_EvtRevBcid<{} {}'.format(clock_cut, clock_cut*2, ecal_bcid_core_cut)
high_cut='sdhcal_EvtRevBcid>={} {}'.format(clock_cut*2, ecal_bcid_core_cut)

ax_deltabcid_cuts_core[1].hist([
    df_common.query(low_cut)['delta_revbcid_corrected'],
    df_common.query(mid_cut)['delta_revbcid_corrected'],
    df_common.query(high_cut)['delta_revbcid_corrected']
    ],
    **hist_options
)

delta_revbcid_core_hist, nbins_delta_revbcid_core, _ = ax_deltabcid_cuts_core[1].hist(
    df_common.query(ecal_bcid_core_cut[5:-1])['delta_revbcid_corrected'],
    bins=hist_options['bins'],
    log=hist_options['log'],
    visible=False
)

# Show legend on right plot
# with plt.xkcd(.5, 100, 2):
#     ax_deltabcid_cuts_core[1].legend(loc='lower right', fontsize='xx-small')

peak_annotations=[]
with plt.xkcd():
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_fringe_hist, bins=nbins_delta_revbcid_fringe, ax=ax_deltabcid_cuts_core[0]))
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_core_hist, bins=nbins_delta_revbcid_core, ax=ax_deltabcid_cuts_core[1]))

resize_labplot(fig_deltabcid_cuts_core, nrows=canvas_options['nrows'], height=h, dpi=canvas_options['dpi'])
fig_deltabcid_cuts_core.savefig('./Figs/delta_bcid_corrected_core_{run_number}.png'.format(run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")
    # fig.savefig('./Figs/{file_title}_{run_number}.pdf'.format(file_title=file_title, run_number=run_number), transparent=False, dpi=300, bbox_inches="tight")


In [ ]:
# Remove all annotation, handy to tweak the style without recomputing the whole hist
display(peak_annotations)
remove_annotations(peak_annotations)
display(peak_annotations)
corrected_peak_options = {
    'prominence': 20,
    'distance': 800
}
with plt.xkcd():
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_hist, bins=nbins_delta_revbcid, ax=ax_deltabcid_cuts[0], peaks_args=corrected_peak_options))
    peak_annotations.extend(draw_peaks(hist=delta_revbcid_corrected_hist, bins=nbins_delta_revbcid_corrected, ax=ax_deltabcid_cuts[1], peaks_args=corrected_peak_options))
fig_deltabcid_cuts.canvas.draw()

With ROOT:

In [ ]:
# import root_numpy as rnp
# import ROOT
total_bins = abs(df_common['delta_revbcid'].min())+abs(df_common['delta_revbcid'].max())
hist_bins = int(total_bins/bin_size) 

can = ROOT.TCanvas('delta_bcid','delta_bcid',2000,1000)
can.Divide(2)
th1_delta_bcid = ROOT.TH1I('delta_bcid','delta_bcid;delta_bcid (clock)', hist_bins, df_common['delta_bcid'].min(), df_common['delta_bcid'].max())
th1_delta_revbcid = ROOT.TH1I('delta_revbcid','delta_revbcid;delta_revbcid (clock)', hist_bins, df_common['delta_revbcid'].min(), df_common['delta_revbcid'].max())
rnp.fill_hist(th1_delta_bcid, df_common['delta_bcid'].to_numpy())
rnp.fill_hist(th1_delta_revbcid, df_common['delta_revbcid'].to_numpy())
can.cd(1)
th1_delta_bcid.Draw()
can.cd(2)
th1_delta_revbcid.Draw()
can.Update()

spect = ROOT.TSpectrum(10)
sigma = 2
threshold =.2
nPeaks = spect.Search(th1_delta_revbcid,sigma,"new", threshold)
peak_xy_list=[]

for peak in range(nPeaks):
    peak_xy_list.append((int(spect.GetPositionX()[peak]),int(spect.GetPositionY()[peak])))

# ROOT sort the peaks by their y-value, need to reorder along the x axis to compute the distance between them
peak_xy_list= sorted(peak_xy_list,key=lambda x:x[0])
prev_x = None
for n, (x, y) in enumerate(peak_xy_list):
    if prev_x:
        display(abs(int(prev_x-x)))
    display((x,y))
    prev_x = x
can.Draw()



In [ ]:
plt.figure()
th2_eVh_bcid = plt.hist2d(df_common['ecal_bcid'],df_common['sdhcal_EvtRevBcid'],[2000, 2000],density=True) # Density= normalisation
# Need to tweak = show the colorbar
# im = plt.imshow(arr, interpolation="none")
# plt.colorbar(im, use_gridspec=True)
                          

In [ ]:
if not 'bcid_corrected.root' in common_file_name:
    common_file_name = common_file_name.replace('.root','_bcid_corrected.root')
display(common_file_name)
from root_pandas import to_root # Needed to write to root file until uproot implement it
df_common.to_root(common_file_name, key=common_tree_name)

In [ ]:
# display(df_common['delta_revbcid'])
# toto = df_common['delta_revbcid'].to_numpy()
toto = np.histogram(df_common['delta_revbcid'],bins=hist_bins)
# display(toto)
with uproot.recreate(out_file_name.replace('.root', 'toto.root'), compression=uproot.ZLIB(4)) as out_file:
    out_file['toto']=toto

with uproot.open(out_file_name .replace('.root', 'toto.root')) as out_file:
    display(out_file.keys())
#     display(out_file['toto'].show())
#     display(out_file['toto'].pandas())
    display(out_file['toto'].numpy())

    myhist,binedges=out_file['toto'].numpy()
    pdhist=pd.DataFrame(data=myhist, index=binedges[:-1])
    figpd,axpd,_=plt.subplots()
    axpd.hist(pdhist)
    fig.show()
    
#     display(pd.DataFrame.from_records(out_file['toto'].numpy()))
    



In [ ]:
hcalFile.

hcalFile['HitMapPerLayer']['hitMap_Layer0'].__dict__

In [ ]:
# Make the common Tree
for trigger in np.random.randint(0,nTriggers,10):
    print(trigger)
    